In [1]:
# Operating System
import os

# Numpy, Pandas and Scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix, save_npz, load_npz

# Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Model Evaluation
#from evaluation import evaluate

## Step 0: Load the Data

## Step 0.1: Load the Train and Test files

In [2]:
def read_users_history() -> pd.DataFrame:
    """Imports the reviews history for each user.
    
    Returns:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, product_id, rating).
                             
    """
    path = os.path.join('data', 'train_reviews.csv')
    data = pd.read_csv(path).drop(columns='Unnamed: 0')
    return data

complete_data = read_users_history()
data = complete_data[['product_id', 'user_id', 'overall']]
data.head()

,product_id,user_id,overall
0,1881509818,AIXZKN4ACSKI,5
1,1881509818,A1L5P841VIO02V,5
2,1881509818,AB2W04NI4OEAD,4
3,1881509818,A148SVSWKTJKU6,4
4,1881509818,AAAWJ6LW9WMOO,4


In [3]:
# How many ratings do we have in total?
# Tip: The ":," at the end of the f-string adds the thousand separator.
print(f"We have {len(data):,} ratings in total.")

We have 268,169 ratings in total.


In [4]:
def read_test_users() -> pd.DataFrame:
    """Imports the list of users for which we need to predict.
    
    Returns:
        users_to_pred (pd.DataFrame): DataFrame with the users for which we will recommend products. 
    """

    path = os.path.join('data', 'test_users.csv')
    users_to_pred_ = pd.read_csv(path)
    users_to_pred_.columns = ['users to recommend products']
    return users_to_pred_


users_to_pred = read_test_users()
users_to_pred.head()

,users to recommend products
0,A0029274J35Q1MYNKUWO
1,A0103849GBVWICKXD4T6
2,A01685981QK9IX1Q16YZY
3,A02904661A62AP64S46MT
4,A036147939NFPC389VLK


In [5]:
# For how many users are we recommending stuff?
print(f"We have {len(users_to_pred)} users in need for better products!")

We have 9225 users in need for better products!


## Step 0.2: Compare the train and test files

In [6]:
# And for how many users we already know something?
# For the 9225 users in our test set, we have data for 8343 in the original reviews history.
# For the remaining 882 we will have to use non-personalized strategies.
print(users_to_pred.isin(data.user_id.values).sum())

users to recommend products    8343
dtype: int64


In [7]:
def get_indices_from_users_to_pred(users_to_pred: pd.DataFrame, data: pd.DataFrame):
    """Get the indices of users_to_pred for which we have data and for which we don't.
    
    Args:
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        data (pd.DataFrame): Original of reviews history for the users.
        
    Returns:
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
        index_users_not_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's without training data.
        
    """
    index_users_in_data = users_to_pred[users_to_pred.isin(data.user_id.values).values].index
    index_users_not_in_data = users_to_pred[~users_to_pred.isin(data.user_id.values).values].index
    
    return index_users_in_data, index_users_not_in_data


index_users_in_data, index_users_not_in_data = get_indices_from_users_to_pred(users_to_pred, data)

In [8]:
# For further inspection, we advise you to look at the objects themselves.
print(f"The index for users which we have training data has length of {len(index_users_in_data)}.")
print(f"The index for users which we don't have training data has length of {len(index_users_not_in_data)}.")

The index for users which we have training data has length of 8343.
The index for users which we don't have training data has length of 882.


In [9]:
def get_users_to_pred_by_index(users_to_pred, index_users_in_data):
    """DataFrame with user_id's in test set for which we have training data.

    Args: 
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
    Returns:
        users_in_data (pd.DataFrame): Dataframe containing the list of user_id's with training data.
    
    """
    return users_to_pred.iloc[index_users_in_data].reset_index(drop=True)

# Get the test users with training data
test_users_in_data = get_users_to_pred_by_index(users_to_pred, index_users_in_data)
print(test_users_in_data.shape)
test_users_in_data.head()

(8343, 1)


,users to recommend products
0,A0029274J35Q1MYNKUWO
1,A0103849GBVWICKXD4T6
2,A01685981QK9IX1Q16YZY
3,A02904661A62AP64S46MT
4,A036147939NFPC389VLK


In [10]:
# Get the test users without training data
test_users_not_in_data = get_users_to_pred_by_index(users_to_pred, index_users_not_in_data)
print(test_users_not_in_data.shape)
test_users_not_in_data.head()

(882, 1)


,users to recommend products
0,A04904273OLXHXFW1AQK2
1,A100US3LDAJU51
2,A10885J2DS1NQ6
3,A108M7R9UBH5LT
4,A10W7NGRNY5GTR


## Step 0.3: Create the Ratings matrix

In [11]:
def make_ratings(data: pd.DataFrame) -> csr_matrix:
    """Creates the ratings matrix of reviews history.
    
    Creates the ratings matrix from the reviews history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  reviews history for the users.
        
    Returns:
        ratings (csr_matrix): ratings matrix with shape (n_users, n_items)
        
    TODO:
        * Add the shape as an optional parameter.
        
    """
    # You have probably seen this before
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape).tocsr()
    
    return R_


R = make_ratings(data)
R

<18264x34713 sparse matrix of type '<class 'numpy.int64'>'
	with 268169 stored elements in Compressed Sparse Row format>

In [12]:
# Just for mental (in)sanity, let's match the info of the matrix to what is printed in the previous cell.
print(f"The shape is {R.shape}")
print(f"The dtypes of the elements are {R.dtype}")
print(f"The number of stored elements is {R.nnz}")
print(f"The type of the matrix is {type(R)}")

The shape is (18264, 34713)
The dtypes of the elements are int64
The number of stored elements is 268169
The type of the matrix is <class 'scipy.sparse.csr.csr_matrix'>


In [13]:
# Let's store a Series with the unique user id's that we have in the original data.
def get_unique_users(data):
    """Get unique users in training data.
    
    Args:
        data (pd.DataFrame):  reviews history for the users.
        
    Returns:
        unique_users (pd.DataFrame): DataFrame of one column with unique users in training data.
    
    """
    return pd.DataFrame(np.unique(data.iloc[:, 1].values), columns=["users to recommend products"])


unique_users_training_data = get_unique_users(data)
unique_users_training_data.head()

,users to recommend products
0,A00046902LP5YSDV0VVNF
1,A0029274J35Q1MYNKUWO
2,A004507634IQ8TNQ3YTUI
3,A005011233SVRED9Q0VY0
4,A00676013IBZZQ3UYWK47


## Step 0.4: Record the indices of users

In [14]:
def get_indices_in_ratings_for_test_users_in_data(data: pd.DataFrame, test_users_in_data: pd.DataFrame):
    """Returns the index of the ratings matrix for the test users for which we have training data.
    
    Args:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user_id, product_id, rating).
                             
        test_users_in_data (pd.DataFrame): DataFrame containing the list of test users for which we have training data.
        
    Returns:
        indices_ratings_tests_users_in_data (np.array): Indices of users in test set with training data for ratings matrix.
    
    """
    unique_users = get_unique_users(data).iloc[:, 0]
    indices_ratings_tests_users_in_data = unique_users[unique_users.isin(test_users_in_data.iloc[:, 0])].index.to_numpy() 
    return indices_ratings_tests_users_in_data

indices_ratings_tests_users_in_data = get_indices_in_ratings_for_test_users_in_data(data, test_users_in_data)

In [15]:
len(indices_ratings_tests_users_in_data)

8343

In [16]:
print(f"As expected, the length of the indices should be {len(indices_ratings_tests_users_in_data)}, matching the number of users in test set with training data.")

As expected, the length of the indices should be 8343, matching the number of users in test set with training data.


## Step 1: Understand your data flow

In [17]:
def make_ratings(data: pd.DataFrame, shape: tuple = None) -> csr_matrix:
    """Creates the ratings matrix of reviews history with optional shape
    
    2nd version of the make_ratings which handles the previous TODO.
    Creates the ratings matrix from the reviews history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  Reviews history for the users.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from data argument.
        
    Returns:
        ratings (csr_matrix): Ratings matrix with shape (n_users, n_items).
    
    """
    # You have probably seen this before
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    if shape == None:
        shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape).tocsr()
    return R_


R = make_ratings(data)
R

<18264x34713 sparse matrix of type '<class 'numpy.int64'>'
	with 268169 stored elements in Compressed Sparse Row format>

In [18]:
# Percentage of reviews history used for validation.
test_size = 0.2

def make_train_val_split(data: pd.DataFrame, test_size : float = 0.2, shape: tuple = None):
    """Split the data into train and validation and returns the ratings matrixes accordingly.
    
    Args:
        data (pd.DataFrame): Reviews history for the users.
        test_size (float): Percentage of reviews history used for validation.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from 'data' argument.
    
    Returns:
        ratings_train (csr_matrix): Ratings matrix for train.
        ratings_val (csr_matrix): Ratings matrix for validation.
    
    """
    train_data, val_data = train_test_split(data, test_size=test_size, random_state=8)
    R_train = make_ratings(train_data, shape=shape)
    R_val = make_ratings(val_data, shape=shape)
    return R_train, R_val

ratings_train, ratings_val = make_train_val_split(data, test_size=test_size, shape=R.shape)

In [19]:
# After the train/validation split, let's compare the number of ratings available in each matrix.
print(f"After the split we have {ratings_train.nnz:,} ratings in the train set and {ratings_val.nnz:,} ratings in the validation set.")

After the split we have 214,535 ratings in the train set and 53,634 ratings in the validation set.


## Step 2: Non-Personalized

In [20]:
def get_most_rated(ratings: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items in a ratings matrix.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    def is_rating(R_: csr_matrix) -> csr_matrix:
        """Returns a sparse matrix of booleans 
        
        Args:
            R_ (csr_matrix): A sparse ratings matrix
            
        Returns:
            is_rating (csr_matrix): A sparse matrix of booleans.
        """
        return R_ > 0
    
    def count_ratings(R_: csr_matrix):
        """Returns an array with the count of ratings
        
        The attribute ".A1" of a numpy matrix returns self as a flattened ndarray.
        
        Args:
            R_ (csr_matrix): A sparse matrix of booleans
        
        Returns:
            count_ratings (np.darray): Count of ratings by item
        """
        return R_.sum(axis=0).A1
    
    ratings_ = is_rating(ratings)
    ratings_ = count_ratings(ratings_)
    return np.negative(ratings_).argsort()[:n]


non_pers_most_rated = get_most_rated(ratings_train, 50)
non_pers_most_rated

array([24664, 31755, 31291, 17681, 10886, 31558, 10589,  9142, 17662,
       15908, 29565,  7143, 17719,  6946, 14150, 16654, 28800, 20460,
       24323,  1379, 29871, 27513, 21372, 26787, 19660,  2659, 20202,
       20493, 14387, 25737, 22901, 19670, 15292,  4134, 23029, 18960,
       27654, 29774, 19990,  3663, 21384, 24826, 23709, 34175,  9817,
        3703, 16156, 25856,  9539, 33804], dtype=int64)

In [21]:
def convert_non_pers_recommendations_to_df(non_pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the non-personalized most rated to an DataFrame with the users and the recommendations.
    We will basically repeat the non_pers_recs array for the number of users in need.
    
    Args:
        non_pers_recs (np.array): Array of indices for the best non-personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    non_pers_df = pd.DataFrame(np.zeros((len(users_to_pred), 1), dtype=non_pers_recs.dtype) + non_pers_recs)
    non_pers_df = pd.concat([users_to_pred, non_pers_df], axis=1)
    non_pers_df = non_pers_df.set_index("users to recommend products")
    
    return non_pers_df


non_pers_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, unique_users_training_data)
non_pers_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
users to recommend products,,,,,,,,,,,,,,,,,,,,,
A00046902LP5YSDV0VVNF,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A0029274J35Q1MYNKUWO,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A004507634IQ8TNQ3YTUI,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A005011233SVRED9Q0VY0,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A00676013IBZZQ3UYWK47,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804


In [22]:
def create_dict_preds(preds_df: pd.DataFrame) -> dict:
    """Convert the predictions DataFrame (index:users -> columns: items) to a dictionary of key (user->list of items).
    
    Args: 
        preds_df (pd.DataFrame): DataFrame containing the users and the ordered predictions.
        
    Returns:
        preds_dict (dict): Dict of (user_id: list of items) used for evaluating the performance.
    
    """
    return {preds_df.index[i]: preds_df.values[i].tolist() for i in range(len(preds_df))}


non_pers_dict = create_dict_preds(non_pers_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(non_pers_dict.items())[0:1])

{'A00046902LP5YSDV0VVNF': [24664,
  31755,
  31291,
  17681,
  10886,
  31558,
  10589,
  9142,
  17662,
  15908,
  29565,
  7143,
  17719,
  6946,
  14150,
  16654,
  28800,
  20460,
  24323,
  1379,
  29871,
  27513,
  21372,
  26787,
  19660,
  2659,
  20202,
  20493,
  14387,
  25737,
  22901,
  19670,
  15292,
  4134,
  23029,
  18960,
  27654,
  29774,
  19990,
  3663,
  21384,
  24826,
  23709,
  34175,
  9817,
  3703,
  16156,
  25856,
  9539,
  33804]}

### Step 2.2 - Creating the Ground Truth

In [23]:
def get_y_true(R_val_, users_to_pred, n=50):
    """Get the ground truth (best recommendations) of the users in the validation set.
    
    Args:
        R_val_ (csr_matrix): Validation set ratings matrix.
        n (int): Number of top-n items.
        
    Returns:
        y_true_df (pd.DataFrame): DataFrame which returns the y_true items.
        
    """
    top_from_R_val = pd.DataFrame(np.negative(R_val_).toarray().argsort()[:, :n])
    y_true_df = pd.concat([users_to_pred, top_from_R_val], axis=1)
    y_true_df = y_true_df.set_index("users to recommend products")
    return y_true_df


y_true_df = get_y_true(ratings_val, unique_users_training_data, n=50)
y_true_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
users to recommend products,,,,,,,,,,,,,,,,,,,,,
A00046902LP5YSDV0VVNF,22912.0,4175.0,21268.0,0.0,23141.0,23142.0,23143.0,23144.0,23145.0,23146.0,...,23180.0,23179.0,23178.0,23177.0,23176.0,23175.0,23174.0,23173.0,23172.0,23171.0
A0029274J35Q1MYNKUWO,19863.0,23149.0,23148.0,23147.0,23146.0,23145.0,23144.0,23143.0,23142.0,23141.0,...,23178.0,23177.0,23176.0,23175.0,23174.0,23173.0,23172.0,23171.0,23170.0,23152.0
A004507634IQ8TNQ3YTUI,1335.0,0.0,23149.0,23148.0,23147.0,23146.0,23145.0,23144.0,23143.0,23142.0,...,23178.0,23177.0,23176.0,23175.0,23174.0,23173.0,23172.0,23171.0,23170.0,23120.0
A005011233SVRED9Q0VY0,18341.0,3504.0,10947.0,6196.0,5308.0,22873.0,19267.0,19871.0,14026.0,10390.0,...,23153.0,0.0,23155.0,23183.0,23182.0,23181.0,23180.0,23179.0,23178.0,23177.0
A00676013IBZZQ3UYWK47,21974.0,7005.0,8500.0,1627.0,546.0,23152.0,23150.0,23149.0,23148.0,23141.0,...,23183.0,23182.0,23181.0,23180.0,23179.0,23178.0,23177.0,23176.0,23175.0,23174.0


In [24]:
# Create the dictionary with the ground truth.
y_true_dict = create_dict_preds(y_true_df)

## Evaluate
We will use the Mean Average Precision @ K to evaluate our predictions.

In [ ]:
evaluate(y_true_dict, non_pers_dict)

## Predict 
To submit your predictions, you just need to convert your recommendations to the formatwe have in the `example_output.csv` file.

In [25]:
# Join both dataframes with user_id's
all_test_users = pd.concat([test_users_in_data, test_users_not_in_data]).reset_index(drop=True)

In [26]:
non_pers_test_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, all_test_users)
print(non_pers_test_most_rated_df.shape)
non_pers_test_most_rated_df.head()

(9225, 50)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
users to recommend products,,,,,,,,,,,,,,,,,,,,,
A0029274J35Q1MYNKUWO,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A0103849GBVWICKXD4T6,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A01685981QK9IX1Q16YZY,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A02904661A62AP64S46MT,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A036147939NFPC389VLK,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804


In [42]:
def save_predictions(predictions: pd.DataFrame, output_path: str):
    """Save predictions to csv.
    Saves the predictions into a csv file with the format we need.
    We keep the index since it contains the user ids.
    Args:
        predictions (pd.DataFrame): DataFrame with user_id as index and ordered recommendations in the columns.
        output_path (str): Filepath for the predictions file.
    """
    
    predictions = predictions.copy()
    cols_to_drop = predictions.columns
    predictions['purchases'] = predictions.apply(lambda row: str([str(row[column]) for column in predictions.columns]), axis=1)
    predictions = predictions.drop(columns=cols_to_drop)
    predictions.to_csv(output_path,index_label='user_id')
    print(f"Saved to csv in '{output_path}'.")
    
    
save_predictions(non_pers_test_most_rated_df, os.path.join("data", "test_non_personalized_recommendations.csv"))

Saved to csv in 'data\test_non_personalized_recommendations.csv'.


In [28]:
# Filter the non-personalized recommendations for the users without training data and save
new_users_recommendations = non_pers_most_rated_df.iloc[index_users_not_in_data]
save_predictions(new_users_recommendations, os.path.join("data", "new_users_non_personalized.csv"))

Saved to csv in 'data\new_users_non_personalized.csv'.


## Step 3: Personalized

### Step 3.1 - Collaborative filtering


In [29]:
def make_user_similarities(R_):
    """Creates the user similarities matrix.
    
    Args:
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        user_similarities (csr_matrix): Matrix with user similarities.
    
    """
    return cosine_similarity(R_, dense_output=False)


user_similarities = make_user_similarities(ratings_train)
user_similarities

<18264x18264 sparse matrix of type '<class 'numpy.float64'>'
	with 1780857 stored elements in Compressed Sparse Row format>

In [30]:
def make_user_predictions_collab_filt(S: csr_matrix, R_: csr_matrix):
    """Predict using collaborative filtering.
    
    Args:
        S (csr_matrix): Similarities matrix (tipically using the cosine_similarity).
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        preds (csr_matrix): Predictions matrix.
    
    """
    weighted_sum = np.dot(S, R_)
    
    # We use the absolute value to support negative similarities.
    # In this particular example there are none.
    sum_of_weights = np.abs(S).sum(axis=1)
    
    preds = weighted_sum / sum_of_weights
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return csr_matrix(preds)
 

collab_filt_user_preds = make_user_predictions_collab_filt(user_similarities, ratings_train)
collab_filt_user_preds

C:\Users\Mayra\Anaconda3\envs\HCKT05\lib\site-packages\scipy\sparse\base.py:596: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


<18264x34713 sparse matrix of type '<class 'numpy.float64'>'
	with 51940017 stored elements in Compressed Sparse Row format>

In [31]:
def sparsity(matrix):
    """Calculates the sparsity of a matrix.
    
    Args:
        matrix (csr_matrix): Sparse matrix.
        
    Returns:
        sparsity_ (float): Sparsity percentage (between 0 and 1).
    
    """
    return 1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])


sparsity(collab_filt_user_preds)

0.9180754545069457

In [32]:
def get_most_rated_from_user_preds(user_preds_: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items from the user predictions.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    pred_ = np.negative(user_preds_).toarray()
    return pred_.argsort()[:, :n]


collab_filt_most_rated = get_most_rated_from_user_preds(collab_filt_user_preds, 50)
print(collab_filt_most_rated.shape)
collab_filt_most_rated

(18264, 50)


array([[33485,  1489, 26787, ...,  4530, 25406, 20268],
       [18734, 23029,  6063, ..., 19074,  1578, 20418],
       [10733,  2962, 31855, ..., 22902,  2938, 12101],
       ...,
       [    0, 23149, 23148, ..., 23170, 23120, 23169],
       [    0, 23149, 23148, ..., 23170, 23120, 23169],
       [    0, 23149, 23148, ..., 23170, 23120, 23169]], dtype=int64)

In [33]:
def convert_pers_recommendations_to_df(pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """Converts the personalized most rated to an DataFrame with the users and the recommendations.
    
    Args:
        pers_recs (np.array): Array of indices for the best personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    pers_df = pd.concat([users_to_pred, pd.DataFrame(pers_recs)], axis=1)
    pers_df = pers_df.set_index("users to recommend products")
    
    return pers_df


collab_filt_most_rated_df = convert_pers_recommendations_to_df(collab_filt_most_rated, unique_users_training_data)
collab_filt_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
users to recommend products,,,,,,,,,,,,,,,,,,,,,
A00046902LP5YSDV0VVNF,33485.0,1489.0,26787.0,23339.0,18166.0,19012.0,18753.0,30573.0,25158.0,27680.0,...,15111.0,10137.0,26103.0,16387.0,13926.0,20815.0,2659.0,4530.0,25406.0,20268.0
A0029274J35Q1MYNKUWO,18734.0,23029.0,6063.0,17605.0,7596.0,8342.0,18654.0,10460.0,29976.0,29961.0,...,9206.0,9896.0,15450.0,14645.0,4910.0,23366.0,26445.0,19074.0,1578.0,20418.0
A004507634IQ8TNQ3YTUI,10733.0,2962.0,31855.0,23076.0,30555.0,29790.0,15287.0,7111.0,33201.0,4960.0,...,4644.0,22618.0,26787.0,10496.0,21587.0,16161.0,12747.0,22902.0,2938.0,12101.0
A005011233SVRED9Q0VY0,16022.0,16653.0,24664.0,31291.0,162.0,10400.0,21289.0,20795.0,7596.0,34424.0,...,6516.0,5058.0,1379.0,20255.0,896.0,6697.0,20374.0,32554.0,29689.0,19496.0
A00676013IBZZQ3UYWK47,18821.0,20040.0,2542.0,4264.0,13732.0,6169.0,32663.0,8969.0,1260.0,7143.0,...,16302.0,10088.0,7586.0,16220.0,31678.0,15382.0,3703.0,30939.0,31821.0,10458.0


In [34]:
collab_filt_dict = create_dict_preds(collab_filt_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(collab_filt_dict.items())[0:1])

{'A00046902LP5YSDV0VVNF': [33485.0,
  1489.0,
  26787.0,
  23339.0,
  18166.0,
  19012.0,
  18753.0,
  30573.0,
  25158.0,
  27680.0,
  21174.0,
  9030.0,
  27031.0,
  32942.0,
  2297.0,
  19674.0,
  17699.0,
  22477.0,
  13838.0,
  3080.0,
  25658.0,
  30236.0,
  21221.0,
  2888.0,
  34175.0,
  1479.0,
  2984.0,
  16291.0,
  17827.0,
  5428.0,
  16553.0,
  31558.0,
  18768.0,
  15621.0,
  7506.0,
  16568.0,
  28308.0,
  9182.0,
  30255.0,
  32479.0,
  15111.0,
  10137.0,
  26103.0,
  16387.0,
  13926.0,
  20815.0,
  2659.0,
  4530.0,
  25406.0,
  20268.0]}

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [ ]:
evaluate(y_true_dict, collab_filt_dict)

## Predict 
To submit your predictions, you just need to convert your recommendations to the format we have in the `example_output.csv` file.

In [35]:
# Filter the collaborative filtering most rated DataFrame  using the test_users_in_data mask.
collab_filt_most_rated_in_data_df = collab_filt_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(collab_filt_most_rated_in_data_df.shape)
collab_filt_most_rated_in_data_df.head()

(8343, 50)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
users to recommend products,,,,,,,,,,,,,,,,,,,,,
A0029274J35Q1MYNKUWO,18734.0,23029.0,6063.0,17605.0,7596.0,8342.0,18654.0,10460.0,29976.0,29961.0,...,9206.0,9896.0,15450.0,14645.0,4910.0,23366.0,26445.0,19074.0,1578.0,20418.0
A0103849GBVWICKXD4T6,21658.0,27486.0,25014.0,12962.0,1861.0,25740.0,32239.0,31715.0,15204.0,25060.0,...,12250.0,14651.0,16912.0,16088.0,10856.0,4290.0,15453.0,30582.0,14289.0,197.0
A01685981QK9IX1Q16YZY,33541.0,27465.0,15321.0,3545.0,25586.0,30363.0,21780.0,13933.0,33336.0,11838.0,...,24237.0,11275.0,13937.0,16464.0,20591.0,13160.0,14748.0,18357.0,23662.0,4666.0
A02904661A62AP64S46MT,27031.0,24664.0,31558.0,12058.0,16557.0,24215.0,14206.0,32176.0,15072.0,9123.0,...,9916.0,4875.0,3703.0,28243.0,28813.0,24960.0,12670.0,6533.0,30433.0,1444.0
A036147939NFPC389VLK,18960.0,3184.0,20202.0,15971.0,815.0,28325.0,20460.0,21799.0,24764.0,5523.0,...,12894.0,25860.0,31814.0,19967.0,22091.0,16969.0,17437.0,32036.0,23171.0,4854.0


In [36]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

(882, 50)


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
users to recommend products,,,,,,,,,,,,,,,,,,,,,
A0107455RVRPB9MY9CFV,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A02752563I3MJ8X8ILTOC,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A038510724QUZG0K4HFMG,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A0387429WAPYLMI7N41V,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804
A100WO06OQR8BQ,24664,31755,31291,17681,10886,31558,10589,9142,17662,15908,...,21384,24826,23709,34175,9817,3703,16156,25856,9539,33804


In [37]:
collab_filt_most_rated_test_df = pd.concat([collab_filt_most_rated_in_data_df, new_users_recommendations])
print(collab_filt_most_rated_test_df.shape)

(9225, 50)


In [43]:
# Save the collaborative filtering recommendations 
save_predictions(collab_filt_most_rated_test_df, os.path.join("data", "collab_filt_recommendations.csv"))

Saved to csv in 'data\collab_filt_recommendations.csv'.


### Step 3.2 - Content-based recommendations

In [ ]:
# If you had a FileNotFoundError, make sure you have unziped the `song_tag.zip` file.
def read_tags():
    """Import the song tags file.
    
    Returns:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        
    """
    path = os.path.join('data', 'song_tag.csv')
    data = pd.read_csv(path)
    return data


tags = read_tags()
print(tags.shape)
tags.head()

In [ ]:
def read_products():
    """Import the products file.
    
    Returns:
        songs (pd.DataFrame): DataFrame containing the product_id #and the corresponding product_index.
    
    """
    path = os.path.join('data', 'songs.txt')
    data = pd.read_csv(path, names=['song_id', 'song_index'], sep=' ')
    return data


songs= read_songs()
print(songs.shape)
songs.head()

In [ ]:
def merge_tags_songs(tags, songs, data):
    """Join the tags, songs and ratings data sources into a single frame).
    
    Args:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
        
    Returns:
        tags_cleaned (pd.DataFrame): Matches the song_id to the tag_index with a corresponding value.
    
    """
    tags_cleaned = (tags.merge(songs, how='left', on='song_index')
                        .merge(data, how='right', on='song_id')
                   )[['song_id', 'tag_index', 'val']]
    return tags_cleaned

tags_cleaned = merge_tags_songs(tags, songs, data)
print(tags_cleaned.shape)
tags_cleaned.head()

In [ ]:
# Get the number of unique song_id's for which we have tags. 
tags_cleaned.song_id.nunique()

In [ ]:
def make_item_profiles(data):
    """Creates the item profiles matrix.
    
    Args:
        data (pd.DataFrame): DataFrame containing the (rows, columns, values) for the ratings matrix.
                             In this case, we have (song_id, tag_index, value).
    
    Returns:
        item_profiles (csr_matrix): Item profiles matrix. Items as rows, tags as columns.
    
    """
    items, item_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    tags, tag_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(items), len(tags))

    item_profiles_ = coo_matrix((values, (item_pos, tag_pos)), shape=shape).tocsr()
    return item_profiles_


item_profiles = make_item_profiles(tags_cleaned)
item_profiles

In [ ]:
def make_user_profiles(R_, item_profiles_):
    """Calculate the user profiles with the items.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        
    Returns:
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
        
    """
    return np.dot(R_, item_profiles_)


user_profiles = make_user_profiles(ratings_train, item_profiles)
user_profiles

In [ ]:
def make_user_predictions_content(R_, item_profiles_, user_profiles_):
    """Produces content-based predictions.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
       
    Returns:
        preds (csr_matrix): Predictions of ratings using content-based recommendations.
    
    """
    
    # Since we are not looking at the values inbetween the cosine_similarity and excluding previously rated items,
    # we can define dense_output=False to speed up.
    preds = cosine_similarity(user_profiles_, item_profiles_, dense_output=False)
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return preds


content_user_preds = make_user_predictions_content(ratings_train, item_profiles, user_profiles)
content_user_preds

In [ ]:
content_most_rated = get_most_rated_from_user_preds(content_user_preds, 50)
print(content_most_rated.shape)
content_most_rated

In [ ]:
content_most_rated_df = convert_pers_recommendations_to_df(content_most_rated, unique_users_training_data)
print(content_most_rated_df.shape)
content_most_rated_df.head()

In [ ]:
content_dict = create_dict_preds(content_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(content_dict.items())[0:2])

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [ ]:
evaluate(y_true_dict, content_dict)

## Predict 
To submit your predictions, you just need to convert your recommendations to the formatwe have in the `example_output.csv` file.

In [ ]:
content_most_rated_in_data_df = content_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(content_most_rated_in_data_df.shape)
content_most_rated_in_data_df.head()

In [ ]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

In [ ]:
content_most_rated_test_df = pd.concat([content_most_rated_in_data_df, new_users_recommendations])
print(content_most_rated_test_df.shape)

In [ ]:
# Save the content-based filtering recommendations 
save_predictions(content_most_rated_test_df, os.path.join("data", "content_recommendations.csv"))

## Final Step: Merge of all recommendations

In [44]:
print(new_users_recommendations.shape[0])
print(collab_filt_most_rated_test_df.shape[0])

882
9225


In [ ]:
# Save the recommendations for all the users
save_predictions(content_most_rated_test_df, os.path.join("data", "content_recommendations.csv"))